In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from wordcloud import WordCloud
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.sparse import save_npz
import warnings


In [ ]:
dataset=pd.read_csv("/content/netflix_titles.csv")
dataset.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [ ]:
dataset.isnull().sum()


,0
show_id,0
type,0
title,0
director,2634
cast,825
country,831
date_added,10
release_year,0
rating,4
duration,3


In [ ]:
new_df=dataset.fillna(method="ffill")
data_new=new_df.fillna(method="bfill")
data_new.head()

/tmp/ipython-input-5-1827545017.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_df=dataset.fillna(method="ffill")
/tmp/ipython-input-5-1827545017.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data_new=new_df.fillna(method="bfill")


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,Kirsten Johnson,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",South Africa,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",South Africa,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,Julien Leclercq,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
df=data_new[["show_id","type","title","director","cast","country","release_year","rating","listed_in"]]
df.head()

,show_id,type,title,director,cast,country,release_year,rating,listed_in
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",United States,2020,PG-13,Documentaries
1,s2,TV Show,Blood & Water,Kirsten Johnson,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries"
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",South Africa,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act..."
3,s4,TV Show,Jailbirds New Orleans,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",South Africa,2021,TV-MA,"Docuseries, Reality TV"
4,s5,TV Show,Kota Factory,Julien Leclercq,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ..."


In [ ]:
df['cast'] = df['cast'].apply(lambda x: x.replace(" ", ""))
df['director'] = df['director'].apply(lambda x: x.replace(" ", ""))
df['listed_in'] = df['listed_in'].apply(lambda x: x.replace(" ", ""))


/tmp/ipython-input-10-3490109751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cast'] = df['cast'].apply(lambda x: x.replace(" ", ""))
/tmp/ipython-input-10-3490109751.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['director'] = df['director'].apply(lambda x: x.replace(" ", ""))
/tmp/ipython-input-10-3490109751.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

In [ ]:
df.head()

,show_id,type,title,director,cast,country,release_year,rating,listed_in
0,s1,Movie,Dick Johnson Is Dead,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",United States,2020,PG-13,Documentaries
1,s2,TV Show,Blood & Water,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",South Africa,2021,TV-MA,"InternationalTVShows,TVDramas,TVMysteries"
2,s3,TV Show,Ganglands,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy,NabihaAkka...",South Africa,2021,TV-MA,"CrimeTVShows,InternationalTVShows,TVAction&Adv..."
3,s4,TV Show,Jailbirds New Orleans,JulienLeclercq,"SamiBouajila,TracyGotoas,SamuelJouy,NabihaAkka...",South Africa,2021,TV-MA,"Docuseries,RealityTV"
4,s5,TV Show,Kota Factory,JulienLeclercq,"MayurMore,JitendraKumar,RanjanRaj,AlamKhan,Ahs...",India,2021,TV-MA,"InternationalTVShows,RomanticTVShows,TVComedies"


In [ ]:
df.isnull().sum()

,0
show_id,0
type,0
title,0
director,0
cast,0
country,0
release_year,0
rating,0
listed_in,0


In [ ]:
df_count = df["type"].value_counts()
df_count1 = df["listed_in"].value_counts().head()
df_count.head()


,count
type,
Movie,6131
TV Show,2676


In [ ]:
df_count1 = df["listed_in"].value_counts().head()
df_count1

,count
listed_in,
"Dramas,InternationalMovies",362
Documentaries,359
Stand-UpComedy,334
"Comedies,Dramas,InternationalMovies",274
"Dramas,IndependentMovies,InternationalMovies",252


In [ ]:
fig = go.Figure(data=[go.Pie(labels=df_count.index, values=df_count.values)])
fig.update_layout(
    plot_bgcolor='rgb(100,100,100)',
    paper_bgcolor='rgb(15,15,15)',
    width=500,
    height=600,
    font_color='white',
    title='Distribution of C. Types',
)
fig.update_traces(marker=dict(colors=['red']))
fig.show()

In [ ]:
top_countries = df['country'].value_counts().head(8)

top_countries

,count
country,
United States,3095
India,1071
United Kingdom,465
Japan,270
South Korea,218
Canada,189
Spain,161
France,136


In [ ]:
country_counts = df['country'].value_counts()

data = pd.DataFrame({'Country': country_counts.index, 'Movie Count': country_counts.values})

fig = px.choropleth(data_frame=data, locations='Country', locationmode='country names',
                    color='Movie Count', title='Number of Movies Released By Country',
                    color_continuous_scale='Reds', range_color=(0, data['Movie Count'].max()),
                    labels={'Movie Count': 'Number of Movies'})
fig.update_layout(
    plot_bgcolor='rgb(17, 17, 17)',
    paper_bgcolor='rgb(17, 17, 17)',
    font_color='red'
)
fig.show()

In [ ]:
df.head(2)

,show_id,type,title,director,cast,country,release_year,rating,listed_in
0,s1,Movie,Dick Johnson Is Dead,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",United States,2020,PG-13,Documentaries
1,s2,TV Show,Blood & Water,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",South Africa,2021,TV-MA,"InternationalTVShows,TVDramas,TVMysteries"


In [ ]:
df["tags"]=df["director"]+df["cast"]+df["listed_in"]+df["country"]+df["title"]


/tmp/ipython-input-24-777653693.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
df.head(2)

,show_id,type,title,director,cast,country,release_year,rating,listed_in,tags
0,s1,Movie,Dick Johnson Is Dead,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",United States,2020,PG-13,Documentaries,"KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan..."
1,s2,TV Show,Blood & Water,KirstenJohnson,"AmaQamata,KhosiNgema,GailMabalane,ThabangMolab...",South Africa,2021,TV-MA,"InternationalTVShows,TVDramas,TVMysteries","KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan..."


In [ ]:
new_df = df[["show_id", "type", "release_year", "title", "rating","tags"]]

In [ ]:
new_df.head()

,show_id,type,release_year,title,rating,tags
0,s1,Movie,2020,Dick Johnson Is Dead,PG-13,"KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan..."
1,s2,TV Show,2021,Blood & Water,TV-MA,"KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan..."
2,s3,TV Show,2021,Ganglands,TV-MA,"JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ..."
3,s4,TV Show,2021,Jailbirds New Orleans,TV-MA,"JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ..."
4,s5,TV Show,2021,Kota Factory,TV-MA,"JulienLeclercqMayurMore,JitendraKumar,RanjanRa..."


In [ ]:
import pandas as pd

# Create a copy to avoid SettingWithCopyWarning
en_data = new_df[["show_id", "type", "title", "release_year", "rating", "tags"]].copy()

# Separate titles into movie and show columns
en_data["title_movie"] = en_data.apply(lambda x: x["title"] if x["type"] == "Movie" else None, axis=1)
en_data["title_show"] = en_data.apply(lambda x: x["title"] if x["type"] == "TV Show" else None, axis=1)

# Drop the original title column
en_data.drop("title", axis=1, inplace=True)

# Remove rows where both title_movie and title_show are None
en_data = en_data[(en_data["title_movie"].notnull()) | (en_data["title_show"].notnull())]

# Display the updated DataFrame
en_data[["show_id", "type", "release_year", "rating", "tags", "title_movie", "title_show"]].head(10)

,show_id,type,release_year,rating,tags,title_movie,title_show
0,s1,Movie,2020,PG-13,"KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan...",Dick Johnson Is Dead,None
1,s2,TV Show,2021,TV-MA,"KirstenJohnsonAmaQamata,KhosiNgema,GailMabalan...",None,Blood & Water
2,s3,TV Show,2021,TV-MA,"JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ...",None,Ganglands
3,s4,TV Show,2021,TV-MA,"JulienLeclercqSamiBouajila,TracyGotoas,SamuelJ...",None,Jailbirds New Orleans
4,s5,TV Show,2021,TV-MA,"JulienLeclercqMayurMore,JitendraKumar,RanjanRa...",None,Kota Factory
5,s6,TV Show,2021,TV-MA,"MikeFlanaganKateSiegel,ZachGilford,HamishLinkl...",None,Midnight Mass
6,s7,Movie,2021,PG,"RobertCullen,JoséLuisUchaVanessaHudgens,Kimiko...",My Little Pony: A New Generation,None
7,s8,Movie,1993,TV-MA,"HaileGerimaKofiGhanaba,OyafunmikeOgunlano,Alex...",Sankofa,None
8,s9,TV Show,2021,TV-14,"AndyDevonshireMelGiedroyc,SuePerkins,MaryBerry...",None,The Great British Baking Show
9,s10,Movie,2021,PG-13,"TheodoreMelfiMelissaMcCarthy,ChrisO'Dowd,Kevin...",The Starling,None


In [ ]:
en_data["tags"].apply(lambda x:" ".join(x))

,tags
0,K i r s t e n J o h n s o n A m a Q a m a t a ...
1,K i r s t e n J o h n s o n A m a Q a m a t a ...
2,J u l i e n L e c l e r c q S a m i B o u a j ...
3,J u l i e n L e c l e r c q S a m i B o u a j ...
4,J u l i e n L e c l e r c q M a y u r M o r e ...
...,...
8802,D a v i d F i n c h e r M a r k R u f f a l o ...
8803,D a v i d F i n c h e r M a r k R u f f a l o ...
8804,R u b e n F l e i s c h e r J e s s e E i s e ...
8805,"P e t e r H e w i t t T i m A l l e n , C o u ..."


In [ ]:
en_data["tags"][0]

'KirstenJohnsonAmaQamata,KhosiNgema,GailMabalane,ThabangMolaba,DillonWindvogel,NatashaThahane,ArnoGreeff,XolileTshabalala,GetmoreSithole,CindyMahlangu,RyleDeMorny,GreteliFincham,SelloMaakeKa-Ncube,OdwaGwanya,MekailaMathys,SandiSchultz,DuaneWilliams,ShamillaMiller,PatrickMofokengDocumentariesUnited StatesDick Johnson Is Dead'

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000,stop_words="english")



In [ ]:
cv.fit_transform(en_data["tags"]).toarray().shape

(8807, 5000)

In [ ]:
vectors=cv.fit_transform(en_data["tags"]).toarray()
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
cv.get_feature_names_out()

array(['10', '100', '101', ..., 'ólafurdarriólafsson', 'özgeborak',
       'özkanuğur'], dtype=object)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
similarity

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
def recommend(title_movie):
    movie_index = en_data[en_data['title_movie'] == title_movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movie_list:
        print(en_data.iloc[i[0]].title_movie)

In [ ]:
recommend("Je Suis Karl")

Sarajevo
Dries
None
Dance Academy: The Comeback
Blessed Benefit


In [ ]:
def recommend(title_show):
    show_index = en_data[en_data['title_show'] == title_show].index[0]
    distances = similarity[show_index]
    show_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in show_list:
        print(en_data.iloc[i[0]].title_show)

In [ ]:
recommend("Blood & Water")

Girl from Nowhere
Katla
Paranormal
Baby
Curon


In [ ]:
import pickle

In [ ]:
pickle.dump(en_data,open("netflix_data.pkl","wb"))

In [ ]:
pickle.dump(en_data.to_dict(),open("netflix_dict.pkl","wb"))

In [ ]:
pickle.dump(similarity, open("similarity.pkl", "wb"))

In [ ]:
from google.colab import files
files.download("similarity.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def recommend(title_movie):
    movie_index = en_data[en_data['title_movie'] == title_movie].index[0]
    distances = similarity[movie_index]
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    for i in movie_list:
        print(i)

In [ ]:
recommend("Je Suis Karl")


(5633, np.float64(0.7071067811865475))
(6649, np.float64(0.7071067811865475))
(1915, np.float64(0.4999999999999999))
(6557, np.float64(0.4999999999999999))
(5222, np.float64(0.408248290463863))
